# Fake News Detection

## Inspiration
With every news channel constantly wanting to attract more and more consumers on their page, it becomes increasingly hard to determine which news articles are real and which are seeking clicks through false headlines and false information. I personally do not want to be misled into believing something that is not true so I wanted to create an app that would tell me whether certain news is true or not.

We see the consequences of fake news in our culture everyday. A good example would be fake news about celebrities that could ruin their image.

In [1]:
# import necessary libraries to work with data and nlp models

import tensorflow
import torch
from transformers import pipeline
import pandas as pd
import numpy as np

In [3]:
# read in the data
all_data_news = pd.read_csv('WELFake_Dataset.csv', index_col=0)
# label: 0 = fake, 1 = real
all_data_news.head(2)

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1


## Plan
We will be using Hugging Face, an AI company, that offers pretrained models (which includes NLP models), to detect fake news.

**Why not train our own model?** An NLP model requires extensive data which often takes days to train, and takes up a lot of space. It is also unethical to train your own data when there are pretrained models which already exist.